# Import RPB model along with other utility functions

In [1]:
from idaes.core import FlowsheetBlock
from idaes.models.unit_models import Feed, Product
from RPB_model import RotaryPackedBed

from pyomo.environ import (
    ConcreteModel,
    SolverFactory,
    TransformationFactory,
    )

import idaes.core.util as iutil
import idaes.core.util.scaling as iscale
from idaes.core.util.model_statistics import degrees_of_freedom
import idaes.logger as idaeslog
from pyomo.environ import units as u
from idaes.core.util.initialization import propagate_state

# from idaes.models_extra.power_generation.properties import FlueGasParameterBlock
from flue_gas_ideal import FlueGasParameterBlock
from idaes.models.properties.modular_properties.base.generic_property import (
    GenericParameterBlock,
)
from idaes.models_extra.power_generation.properties.natural_gas_PR import (
    get_prop,
    EosType,
)

from pyomo.network import Arc

from idaes.core.util.model_diagnostics import DiagnosticsToolbox

import numpy as np

In [2]:
# create Flowsheet block

m = ConcreteModel()
m.fs = FlowsheetBlock(dynamic = False)

In [3]:
# create gas phase properties block

flue_species={"H2O", "CO2", "N2"}
m.fs.gas_props = GenericParameterBlock(
            **get_prop(flue_species, ["Vap"], eos=EosType.IDEAL),
            doc="Flue gas properties",
        )

In [4]:
# create feed and product blocks

m.fs.flue_gas_in = Feed(property_package = m.fs.gas_props)
m.fs.flue_gas_out = Product(property_package = m.fs.gas_props)
m.fs.steam_sweep_feed = Feed(property_package = m.fs.gas_props)
m.fs.regeneration_prod = Product(property_package = m.fs.gas_props)

In [5]:
m.fs.flue_gas_in.properties[0].define_state_vars()

{'flow_mol': <pyomo.core.base.var.ScalarVar at 0x258299e3140>,
 'mole_frac_comp': <pyomo.core.base.var.IndexedVar at 0x25829aca2c0>,
 'temperature': <pyomo.core.base.var.ScalarVar at 0x25829accac0>,
 'pressure': <pyomo.core.base.var.ScalarVar at 0x25829acc900>}

In [6]:
# best discretization

z_init_points=tuple(np.geomspace(0.01, 0.5, 9)[:-1]) + tuple((1 - np.geomspace(0.01, 0.5, 9))[::-1])
o_init_points=tuple(np.geomspace(0.005, 0.1, 8)) + tuple(np.linspace(0.1, 0.995, 10)[1:])

z_nfe=20
o_nfe=20

m.fs.RPB = RotaryPackedBed(
    property_package = m.fs.gas_props,
    z_init_points=z_init_points,
    o_init_points=o_init_points,
    z_nfe=z_nfe,
    o_nfe=o_nfe,
)

In [7]:
# # limited discretization, used for debugging

# m.fs.RPB = RotaryPackedBed(
#     property_package = m.fs.gas_props,
# )

In [8]:
# check degrees of freedom before connecting streams

print("DOF =",degrees_of_freedom(m))

DOF = 30


In [9]:
# add stream connections

m.fs.s01 = Arc(source=m.fs.flue_gas_in.outlet, destination=m.fs.RPB.ads_gas_inlet)
m.fs.s02 = Arc(source=m.fs.RPB.ads_gas_outlet, destination=m.fs.flue_gas_out.inlet)
m.fs.s03 = Arc(source=m.fs.steam_sweep_feed.outlet, destination=m.fs.RPB.des_gas_inlet)
m.fs.s04 = Arc(source=m.fs.RPB.des_gas_outlet, destination=m.fs.regeneration_prod.inlet)

In [10]:
TransformationFactory("network.expand_arcs").apply_to(m)

In [11]:
# check degrees of freedom after applying transformation

print("DOF =",degrees_of_freedom(m))

DOF = 14


In [12]:
# fix streams around RPB

m.fs.RPB.ads_gas_inlet.pressure.fix(1.02*1e5)
m.fs.RPB.ads_gas_outlet.pressure.fix(1.01325*1e5)
m.fs.RPB.ads_gas_inlet.temperature.fix()
# m.fs.RPB.ads.gas_inlet.F_in.fix()
m.fs.RPB.ads_gas_inlet.mole_frac_comp.fix()

m.fs.RPB.des_gas_inlet.pressure.fix(1.05*1e5)
m.fs.RPB.des_gas_outlet.pressure.fix(1.01325*1e5)
m.fs.RPB.des_gas_inlet.temperature.fix()
# m.fs.RPB.des.gas_inlet.F_in.fix()
m.fs.RPB.des_gas_inlet.mole_frac_comp.fix()

In [13]:
# fix design variables of the RPB

m.fs.RPB.ads.Tx.fix()
m.fs.RPB.des.Tx.fix()

m.fs.RPB.w_rpm.fix(0.1)

In [14]:
print("DOF =",degrees_of_freedom(m))

DOF = 0


In [15]:
# load model

iutil.from_json(m, fname="RPB flowsheet 052324.json.gz", gz=True)

{'etime_load_file': 0.10979342460632324,
 'etime_read_dict': 0.29868221282958984,
 'etime_read_suffixes': 0.02291560173034668}

In [ ]:
# initialize feed and product blocks

m.fs.flue_gas_in.initialize()
m.fs.flue_gas_out.initialize()
m.fs.steam_sweep_feed.initialize()
m.fs.regeneration_prod.initialize()

In [ ]:
# Initialize RPB

optarg = {
    # "halt_on_ampl_error": "yes",
    "max_iter": 1000,
    # "bound_push": 1e-22,
    # "mu_init": 1e-3,
    "nlp_scaling_method": "user-scaling",
}

init_points = [1e-10,1e-3,1e-1,0.5,]

m.fs.RPB.initialize(outlvl=idaeslog.DEBUG, optarg=optarg, initialization_points=init_points)

In [16]:
Solver = SolverFactory("ipopt")
Solver.solve(m, tee=True).write()

component keys that are not exported as part of the NL file.  Skipping.
that are not Var, Constraint, Objective, or the model.  Skipping.
Ipopt 3.13.2: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    publicity material resulting from use of the HSL codes within IPOPT must
    con

In [17]:
m.fs.report()


Flowsheet : fs                                                             Time: 0.0
------------------------------------------------------------------------------------
    Stream Table
                                Units          s01        s02        s03        s04   
    Total Molar Flowrate     mole / second     73.318     70.427     173.12     176.58
    Total Mole Fraction N2   dimensionless    0.87000    0.90571  0.0010000 0.00098039
    Total Mole Fraction H2O  dimensionless   0.090000   0.093694    0.99899    0.97940
    Total Mole Fraction CO2  dimensionless   0.040000 0.00059670 1.0000e-05   0.019619
    Temperature                     kelvin     363.00     386.88     393.00     368.85
    Pressure                        pascal 1.0200e+05 1.0132e+05 1.0500e+05 1.0132e+05


In [ ]:
diagtool = DiagnosticsToolbox(m)

In [ ]:
diagtool.report_structural_issues()

In [ ]:
diagtool.report_numerical_issues()

In [ ]:
# save model

iutil.to_json(m, fname="RPB flowsheet 052324.json.gz", gz=True, human_read=False)

In [18]:
m.fs.RPB.report(dof=True)


Unit : fs.RPB                                                              Time: 0.0
    Local Degrees of Freedom: 0
    Total Variables: 22285    Activated Constraints: 21951    Activated Blocks: 7
------------------------------------------------------------------------------------
    Unit Performance

    Variables: 

    Key                        : Value    : Units           : Fixed : Bounds
                 Adsorption Tx :   393.00 :          kelvin :  True : (0, None)
    Adsorption Volume Fraction :  0.75000 :   dimensionless :  True : (0.01, 0.99)
                   CO2 Capture :  0.98567 :   dimensionless : False : (None, None)
    Desorption Volume Fraction :  0.25000 :   dimensionless : False : (0.01, 0.99)
                      Diameter :   10.000 :           meter :  True : (0, None)
                        Length :   3.0000 :           meter :  True : (0.1, 10.001)
           Rotational Velocity : 0.010472 : radian / second :  True : (1e-05, 2)

    Expressions: 

    K

# Creating model for a single side/section of the RPB

In [ ]:
# Create model instance. Currently, mode can be either "adsorption" or "desorption" which sets the initial values and boundary conditions for each case.
# m=RPB_model(mode="adsorption", gas_flow_direction=1)
# or
RPB = RotaryPackedBed()
# RPB.m = Block()
add_single_section_equations(RPB, section_name = "m", mode="adsorption", gas_flow_direction="forward")

# Custom initialization routine. Uses block initialization function.
single_section_init(RPB.m)

In [ ]:
solver = SolverFactory("ipopt")
solver.options = {
    "max_iter": 1000,
    "bound_push": 1e-22,
    "halt_on_ampl_error": "yes",
    "nlp_scaling_method": "user-scaling",
}
solver.solve(RPB, tee=True).write()

In [ ]:
# Some various utility functions to check model performance
evaluate_MB_error(RPB.m)

print(f'CO2 Capture = {RPB.m.CO2_capture[0]():.3}')

In [ ]:
# scaling functions

# check_scaling(m)

# jac, variables, constraints = scaling_script(m)

In [ ]:
plotting(RPB.m)

# Creating a full RPB model

From scratch

In [ ]:
# create pyomo model
RPB = full_model_creation(lean_temp_connection=True, configuration = "counter-current")

In [ ]:
RPB.ads.P.setub(1.26)
RPB.ads.P_in.setub(1.26)

RPB.des.P.setub(1.04)

RPB.L.fix(7.811853)
RPB.ads.theta.fix(0.606758)
RPB.des.P_in.fix(1.034350)
RPB.ads.Tx.fix(347.700154)
RPB.des.Tx.fix(433)
# RPB.ads.w_rpm.fix(0.003502)
RPB.ads.P_in.fix(1.250714)

In [ ]:
# initialize using BlockTriangularizationInitializer() with a list of values for initialization factors within the models
# init_routine_1(RPB, homotopy_points=[1e-5,1e-4,1e-3,1e-2] + np.linspace(0.1,1,5).tolist())
init_routine_1(RPB, homotopy_points=[1e-3] + np.linspace(0.1,1,5).tolist())

In [ ]:
solver = SolverFactory("ipopt")
solver.options = {
    "max_iter": 1000,
    "bound_push": 1e-22,
    # "halt_on_ampl_error": "yes",
}
solver.solve(RPB, tee=True).write()

In [ ]:
init_obj = BlockTriangularizationInitializer()

init_obj.config.block_solver_call_options = {"tee": True}
init_obj.config.block_solver_options = {
    # "halt_on_ampl_error": "yes",
    "max_iter": 1000,
}

# target = 0.003502
targets = [0.1,0.05,0.01,0.005,0.003502]

for target in targets:

    steps = np.linspace(0,1,5)

    points = [(target - RPB.w_rpm())*i + RPB.w_rpm() for i in steps]

    for i in points:
        RPB.w_rpm.fix(i)
        
        init_obj.initialization_routine(RPB)

    # solve using conopt thorugh gams
    results = SolverFactory("gams").solve(
        RPB,
        tee=True,
        keepfiles=True,
        solver="conopt4",
        tmpdir="temp",
        add_options=["gams_model.optfile=1;"],
    )

In [ ]:
get_init_factors(RPB.ads) # these should all be = 1

In [ ]:
report(RPB)

From a previous solution

In [ ]:
# create pyomo model
RPB = full_model_creation(lean_temp_connection=True, configuration = "counter-current")

# load a previous solution and solve
from_json(RPB, fname="base case solution 012424.json.gz", gz=True)

solver = SolverFactory("ipopt")
solver.options = {
    "max_iter": 1000,
    "bound_push": 1e-22,
    "halt_on_ampl_error": "yes",
}
solver.solve(RPB, tee=True).write()

In [ ]:
report(RPB)

In [ ]:
evaluate_MB_error(RPB.ads)
print(' ')
evaluate_MB_error(RPB.des)

In [ ]:
# different scaling functions

# check_scaling(RPB)

# jac, variables, constraints = scaling_script(RPB)

# Optimization

start from initialized model

In [ ]:
# starting from initialized model. Change design to fix capture by freeing up inlet adsorber pressure

RPB.ads.P_in.unfix()
RPB.ads.CO2_capture.fix(0.95)

solver = SolverFactory("ipopt")
solver.options = {
    "max_iter": 1000,
    "bound_push": 1e-22,
    # "halt_on_ampl_error": "yes",
}
solver.solve(RPB, tee=True).write()

In [ ]:
# create regularization parameter for the objective function
RPB.alpha_obj = Param(initialize=0.5, mutable=True)

# add objective
@RPB.Expression()
def obj(RPB):
    return RPB.alpha_obj * RPB.energy_requirement - (1 - RPB.alpha_obj) * RPB.productivity

RPB.objective = Objective(expr=RPB.obj)

RPB.objective.pprint()

In [ ]:
# set bounds for decision variables
RPB.ads.L.setlb(0.01)
RPB.ads.L.setub(40)
RPB.des.L.setlb(0.01)
RPB.des.L.setub(40)
RPB.ads.L.pprint()

In [ ]:
RPB.ads.Tx.setlb(25+273)
RPB.ads.Tx.setub(95+273)
RPB.ads.Tx.pprint()

In [ ]:
RPB.des.Tx.setlb(100+273)
RPB.des.Tx.setub(160+273)
RPB.des.Tx.pprint()

In [ ]:
RPB.ads.P_in.setub(1.5)
RPB.ads.P_in.pprint()
RPB.ads.P.setub(1.5)

In [ ]:
RPB.des.P_in.setub(1.5)
RPB.des.P_in.setlb(1.01325)
RPB.des.P.setub(1.5)
RPB.des.P_in.pprint()

In [ ]:
RPB.ads.w_rpm.setlb(0.00001)
RPB.ads.w_rpm.setub(0.005)
RPB.ads.w_rpm.pprint()

In [ ]:
# free up decision variables (keep des.Tx fixed for first run the free up later)
RPB.ads.L.unfix()
RPB.ads.theta.unfix()
RPB.des.P_in.unfix()
RPB.ads.Tx.unfix()
RPB.des.Tx.unfix()
RPB.ads.w_rpm.unfix()

In [ ]:
degrees_of_freedom(RPB)

In [ ]:
RPB.ads.w_rpm.unfix()

In [ ]:
# solve using conopt thorugh gams
results = SolverFactory("gams").solve(
    RPB,
    tee=True,
    keepfiles=True,
    solver="conopt4",
    tmpdir="temp",
    add_options=["gams_model.optfile=1;"],
)

In [ ]:
# or solve using ipopt
# solver = SolverFactory("ipopt")
# solver.options = {
#     "max_iter": 1000,
#     "bound_push": 1e-8,
#     # "halt_on_ampl_error": "yes",
#     "tol": 1e-4,
#     "max_cpu_time": 5*60,
#     # "mu_strategy": "adaptive",
# }
# solver.solve(RPB, tee=True).write()

In [ ]:
results_df = report(RPB)
results_df

In [ ]:
evaluate_MB_error(RPB.ads)
print(' ')
evaluate_MB_error(RPB.des)

In [ ]:
# custom function using degeneracy hunter. Mainly to see if any variables are pushing their bounds
degen_hunter(RPB)

start from previous optimized case

In [ ]:
# create pyomo model
RPB = full_model_creation(lean_temp_connection=True, configuration = "counter-current")

In [ ]:
# create regularization parameter for the objective function
RPB.alpha_obj = Param(initialize=0.5, mutable=True)

# add objective
@RPB.Expression()
def obj(RPB):
    return RPB.alpha_obj * RPB.energy_requirement - (1 - RPB.alpha_obj) * RPB.productivity

RPB.objective = Objective(expr=RPB.obj)

RPB.objective.pprint()

In [ ]:
# set bounds for decision variables
RPB.ads.L.setlb(0.01)
RPB.ads.L.setub(40)
RPB.des.L.setlb(0.01)
RPB.des.L.setub(40)
RPB.ads.L.pprint()

In [ ]:
RPB.ads.Tx.setlb(25+273)
RPB.ads.Tx.setub(95+273)
RPB.ads.Tx.pprint()

In [ ]:
RPB.des.Tx.setlb(100+273)
RPB.des.Tx.setub(160+273)
RPB.des.Tx.pprint()

In [ ]:
RPB.ads.P_in.setub(1.5)
RPB.ads.P_in.pprint()
RPB.ads.P.setub(1.5)

In [ ]:
RPB.des.P_in.setub(1.5)
RPB.des.P_in.setlb(1.01325)
RPB.des.P.setub(1.5)
RPB.des.P_in.pprint()

In [ ]:
RPB.ads.w_rpm.setlb(0.00001)
RPB.ads.w_rpm.setub(0.1)
RPB.ads.w_rpm.pprint()

In [ ]:
# load a previous solution and solve
from_json(RPB, fname="opt solution 012424.json.gz", gz=True)

In [ ]:
RPB.alpha_obj.pprint()

In [ ]:
# free up decision variables (keep des.Tx fixed for first run the free up later)
RPB.ads.L.unfix()
RPB.ads.theta.unfix()
RPB.des.P_in.unfix()
RPB.ads.Tx.unfix()
RPB.des.Tx.unfix()
RPB.ads.w_rpm.unfix()

In [ ]:
degrees_of_freedom(RPB)

In [ ]:
# solve using conopt thorugh gams
results = SolverFactory("gams").solve(
    RPB,
    tee=True,
    keepfiles=True,
    solver="conopt4",
    tmpdir="temp",
    add_options=["gams_model.optfile=1;"],
)

In [ ]:
report(RPB)

In [ ]:
degen_hunter(RPB)

pareto front generation

In [ ]:
# list of alpha values to use in the objective function
alpha_list=[
    0.0001,
    0.001,
    0.005,
    0.01,
    0.02,
    0.05,
    0.1,
    0.2,
    0.3,
    0.4,
    0.5,
    0.6,
    0.7,
    0.8,
    0.9,
    0.925,
    0.95,
    0.975,
    0.99,
    0.999,
]

In [ ]:
# optimize for every value and store the results
E = []
P = []

for j in alpha_list:
    RPB.alpha_obj = j

    results = SolverFactory("gams").solve(
        RPB,
        tee=True,
        keepfiles=True,
        solver="conopt4",
        tmpdir="temp",
        add_options=["gams_model.optfile=1;"],
    )

    print(f'alpha = {j}, E={RPB.energy_requirement()}, P={RPB.productivity()}')

    E.append(RPB.energy_requirement())
    P.append(RPB.productivity())


In [ ]:
pd.DataFrame({'alpha':alpha_list,'E':E,'P':P})

In [ ]:
plt.scatter(E,P)

# Polishing step simulation and optimization

start from previous optimized case

In [ ]:
# create pyomo model
RPB = full_model_creation(lean_temp_connection=True, configuration = "counter-current")

In [ ]:
# create regularization parameter for the objective function
RPB.alpha_obj = Param(initialize=0.5, mutable=True)

# add objective
@RPB.Expression()
def obj(RPB):
    return RPB.alpha_obj * RPB.energy_requirement - (1 - RPB.alpha_obj) * RPB.productivity

RPB.objective = Objective(expr=RPB.obj)

RPB.objective.pprint()

In [ ]:
# set bounds for decision variables
RPB.ads.L.setlb(0.01)
RPB.ads.L.setub(40)
RPB.des.L.setlb(0.01)
RPB.des.L.setub(40)
RPB.ads.L.pprint()

In [ ]:
RPB.ads.Tx.setlb(25+273)
RPB.ads.Tx.setub(95+273)
RPB.ads.Tx.pprint()

In [ ]:
RPB.des.Tx.setlb(100+273)
RPB.des.Tx.setub(160+273)
RPB.des.Tx.pprint()

In [ ]:
RPB.ads.P_in.setub(1.5)
RPB.ads.P_in.pprint()
RPB.ads.P.setub(1.5)

In [ ]:
RPB.des.P_in.setub(1.5)
RPB.des.P_in.setlb(1.01325)
RPB.des.P.setub(1.5)
RPB.des.P_in.pprint()

In [ ]:
RPB.ads.w_rpm.setlb(0.00001)
RPB.ads.w_rpm.setub(0.1)
RPB.ads.w_rpm.pprint()

In [ ]:
# load a previous solution and solve (will also load the inlet gas feed conc.)
from_json(RPB, fname="polishing step optimized solution 031824.json.gz", gz=True)

In [ ]:
# solve using conopt thorugh gams
results = SolverFactory("gams").solve(
    RPB,
    tee=True,
    keepfiles=True,
    solver="conopt4",
    tmpdir="temp",
    add_options=["gams_model.optfile=1;"],
)

In [ ]:
report(RPB)

# Plotting

In [ ]:
full_contactor_plotting(RPB)

# Save Model

In [ ]:
# save model
to_json(RPB, fname="base case solution 012424.json.gz", gz=True, human_read=False)

# Diagnostics testing

In [ ]:
# iscale.set_scaling_factor(RPB.ads.Tg_out_eq, 0.1)
# iscale.set_scaling_factor(RPB.des.Tg_out_eq, 0.1)

# for z in RPB.ads.z:
#     for o in RPB.ads.o:
#         if 0 < z < 1 and 0 < o < 1:
#             iscale.set_scaling_factor(RPB.des.pde_solidEB[z,o], 1e-2)
#             iscale.set_scaling_factor(RPB.ads.Q_gs_eq[z, o], 0.001)

In [ ]:
# # solve using conopt thorugh gams
# results = SolverFactory("gams").solve(
#     RPB,
#     tee=True,
#     keepfiles=True,
#     solver="conopt4",
#     tmpdir="temp",
#     add_options=["gams_model.optfile=1;"],
# )

In [ ]:
# RPB.ads.L.fix()
# RPB.ads.theta.fix()
# RPB.des.P_in.fix()
# RPB.ads.Tx.fix()
# RPB.des.Tx.fix()
# RPB.ads.w_rpm.fix()

# degrees_of_freedom(RPB) # need dof=0 for diagnostic tools

In [ ]:
diagtool = DiagnosticsToolbox(m)

In [ ]:
diagtool.report_structural_issues()

In [ ]:
diagtool.report_numerical_issues()

In [ ]:
diagtool.display_constraints_with_extreme_jacobians()

In [ ]:
iscale.get_scaling_factor(RPB.ads.Q_gs_eq[0.5,0.005])

In [ ]:
value(RPB.des.Cp_g_out * RPB.des.Tg_out)

In [ ]:
m=RPB.des
z=0.8846928460920032
o=0.005

value((1 - m.eb) * m.rho_sol * m.Cp_sol * m.w * m.dTsdo[z, o])

In [ ]:
print(units.get_units(RPB.productivity))

In [ ]:
from pyomo.util.check_units import assert_units_consistent, assert_units_equivalent, check_units_equivalent

In [ ]:
assert_units_equivalent(RPB.ads.vel0)

In [ ]:
assert_units_equivalent(RPB.ads.Cp_g["CO2", 0.5, 0.1])

In [ ]:
check_units_equivalent(RPB.ads.qCO2_eq[0.5, 0.1])

In [ ]:
assert_units_consistent(RPB.lean_temp_constraint)

In [ ]:
print(units.get_units(RPB.ads.iso_w2[0.5, 0.1]))

In [ ]:
check_scaling(RPB)

In [ ]:
m=RPB.des
z=0.8846928460920032
o=0.005

m.Q_gs[z, o]()

In [ ]:
RPB.ads.Q_gs_eq[0.04336244396414017,0.005]()/RPB.ads.R_HT_gs() * RPB.ads.h_gs[0.04336244396414017,0.005]() * RPB.ads.a_s()